#               Riiid -🔥🔥🔥Model Explainability🔥🔥🔥


## <a name="Riiid! Answer Correctness Prediction">About this Competition</a>

Riiid Labs, an AI solutions provider delivering creative disruption to the education market, empowers global education players to rethink traditional ways of learning leveraging AI. With a strong belief in equal opportunity in education, Riiid launched an AI tutor based on deep-learning algorithms in 2017 that attracted more than one million South Korean students. This year, the company released EdNet, the world’s largest open database for AI education containing more than 100 million student interactions.

In this competition, your challenge is to create algorithms for "Knowledge Tracing," the modeling of student knowledge over time. The goal is to accurately predict how students will perform on future interactions. You will pair your machine learning skills using Riiid’s EdNet data.

Your innovative algorithms will help tackle global challenges in education. If successful, it’s possible that any student with an Internet connection can enjoy the benefits of a personalized learning experience, regardless of where they live. With your participation, we can build a better and more equitable model for education in a post-COVID-19 world.

## Contents
- <a href='#1'>1. Introduction</a>  
- <a href='#2'>2. Data Extraction </a>
- <a href='#3'>3. Glimpse of Data</a>
- <a href='#4'>4. Feature Engineering </a>
- <a href='#5'>5. Baseline Model</a>
- <a href='#9'>6. Model Explainability</a>

# Introduction

The challenge will rely on the world’s largest education dataset, EdNet, which consists of more than 130 million interactions coming from over 780,000 students. EdNet will be offered to top researchers and scientists.

In this competition, we will just try to create an algorithm for "Knowledge Tracing," the modeling of student knowledge over time. our goal is to accurately predict how students will perform on future interactions. If successful, it’s possible that any student with an Internet connection can enjoy the benefits of a personalized learning experience, regardless of where they live.

Our challenge in this competition is to predict whether students are able to answer their next questions correctly.

This competition is similar to Two Sigma competition, so we got test data using special API.

In [ ]:
!pip install datatable

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import datatable as dt
from sklearn import model_selection


import os 

import plotly.express as px
import plotly.graph_objects as go 
from sklearn.model_selection import StratifiedKFold , KFold, RepeatedKFold,GroupKFold , GridSearchCV , train_test_split ,TimeSeriesSplit
from sklearn.metrics import roc_auc_score
color = sns.color_palette()


import multiprocessing as mp 

import optuna # Hyperparameter tuning

import warnings 
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## <a name="dataset_description">Dataset Description</a>: 

The data is images of wheat fields, with bounding boxes for each identified wheat head. Not all images include wheat heads / bounding boxes. The images were recorded in many locations around the world.

The CSV data is simple - the image ID matches up with the filename of a given image, and the width and height of the image are included, along with a bounding box (see below). There is a row in train.csv for each bounding box. Not all images have bounding boxes.

Most of the test set images are hidden. A small subset of test images has been included for your use in writing code.

What am I predicting?
You are attempting to predict bounding boxes around each wheat head in images that have them. If there are no wheat heads, you must predict no bounding boxes.

File details :- 

1.     train.csv - the training data
2.     questions.csv - List question ID in the online education app
3.     lectures.csv - Leture details coved in this file
4.     example_sample_submission.csv - Sample Submission format 



### <a name="target_variable">Target Variable</a>                                        
* __Submission data__  
    user_id & answered_correctly
    
    
    


In [ ]:
data_types = {
    'timestamp': 'int64',
    'user_id': 'int64',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'
}

In [ ]:
%%time
train = dt.fread('/kaggle/input/riiid-test-answer-prediction/train.csv').to_pandas()

In [ ]:
train = train[['timestamp', 'user_id', 'content_id', 'content_type_id','task_container_id', 'answered_correctly','prior_question_elapsed_time', 'prior_question_had_explanation']]

In [ ]:
for column, d_type in data_types.items():
    train[column] = train[column].astype(d_type) 

In [ ]:
train.head()

# Take fraction of data for analysis while submission kindly comment this line 

### Split the training set by 5 parts for Quicker iterations 

In [ ]:
train = train.sample(len(train)//5,random_state=42)

In [ ]:
#train = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', usecols=[1, 2, 3,4,5,7,8,9], dtype={'timestamp': 'int64', 'user_id': 'int32' ,'content_id': 'int16','content_type_id': 'int8','answered_correctly':'int8','prior_question_elapsed_time': 'float32','prior_question_had_explanation': 'boolean','task_container_id': 'int16'})
train = train[train.content_type_id == False]
#arrange by timestamp

train = train.sort_values(['timestamp'], ascending=True)

In [ ]:
! head 10 /kaggle/input/riiid-test-answer-prediction/train.csv

In [ ]:
train.answered_correctly.unique()

### Loading datasets having less memory 

In [ ]:
path = '/kaggle/input'
"""


train = pd.read_csv(f'{path}/riiid-test-answer-prediction/train.csv', low_memory=False, 
                       dtype={'row_id': 'int64',
                              'timestamp': 'int64',
                              'user_id': 'int32',
                              'content_id': 'int16',
                              'content_type_id': 'int8',
                              'task_container_id': 'int16',
                              'user_answer': 'int8',
                              'answered_correctly': 'int8',
                              'prior_question_elapsed_time': 'float32', 
                              'prior_question_had_explanation': 'boolean',
                             }
                      )
"""

test = pd.read_csv(f'{path}/riiid-test-answer-prediction/example_test.csv')
submit = pd.read_csv(f'{path}/riiid-test-answer-prediction/example_sample_submission.csv')
lectures = pd.read_csv(f'{path}/riiid-test-answer-prediction/lectures.csv')

train = train[train.content_type_id == False]


print('Train shapes: ', train.shape)
print('Test shapes: ', test.shape)

# Feature Engineering 

#### "Applied Machine Learning is northing but feature engineering" - Andrew Ng

In [ ]:
results_c = train[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
results_c.columns = ["answered_correctly_content"]

results_u = train[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'sum'])
results_u.columns = ["answered_correctly_user", 'sum']

In [ ]:
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',
                            usecols=[0,1, 3,4],
                            dtype={'question_id': 'int16',
                              'part': 'int8','bundle_id': 'int8','tags': 'str'}
                          )
tag = questions_df["tags"].str.split(" ", n = 10, expand = True) 
tag.columns = ['tags1','tags2','tags3','tags4','tags5','tags6']

questions_df =  pd.concat([questions_df,tag],axis=1)
questions_df['tags1'] = pd.to_numeric(questions_df['tags1'], errors='coerce')
questions_df['tags2'] = pd.to_numeric(questions_df['tags2'], errors='coerce')
questions_df['tags3'] = pd.to_numeric(questions_df['tags3'], errors='coerce')
questions_df['tags4'] = pd.to_numeric(questions_df['tags4'], errors='coerce')
questions_df['tags5'] = pd.to_numeric(questions_df['tags5'], errors='coerce')
questions_df['tags6'] = pd.to_numeric(questions_df['tags6'], errors='coerce')

### Five Point Summary 

In [ ]:
train.describe()

count of content_type_id is less , there are missing values in it , It requires treatment

### --- Start of Sample data analysis ----

### Let understand the data with sample 

To understand data ; Lets dive in.  For User ID - 115 there are multiple content Id
Content type ID - 0 or 1 depend on weather the user watching the lecture or answering the question

For Content type ID - 0 ( User lectures ) , user_answer and answered_correctly data value should be -1 


In [ ]:
train[(train['user_id']==2746)]

In [ ]:

questions_df[questions_df['question_id']==128]

In [ ]:
lectures[lectures['lecture_id']==128]

In [ ]:
print('Total unique users :- ',train['user_id'].nunique())

In [ ]:
df_correct_question_anys = train[train['content_type_id']==0].groupby(['user_id']).agg({'content_id':'count','answered_correctly':'sum'}).reset_index().rename(columns={'content_id':'Total_questions'})

In [ ]:
df_correct_question_anys['answered_wrongly']=df_correct_question_anys['Total_questions'] - df_correct_question_anys['answered_correctly']

In [ ]:
df_correct_question_anys['Pass_pct']= df_correct_question_anys['answered_correctly'] / df_correct_question_anys['Total_questions']
df_correct_question_anys['Fail_pct']= df_correct_question_anys['answered_wrongly'] / df_correct_question_anys['Total_questions']

### --- End of Sample data analysis ----

In [ ]:
train.drop(['timestamp','content_type_id','task_container_id'], axis=1,   inplace=True)

In [ ]:
### We have divided the dataset into 5 parts , If we training with full dataset please uncomment below line 
#X = train.iloc[80000000:,:]

X = train.copy()
X['prior_question_had_explanation'].fillna(False, inplace=True)

X['prior_question_had_explanation'] = X['prior_question_had_explanation'].fillna(X['prior_question_had_explanation'].mode()[0])
X['prior_question_elapsed_time'] = X['prior_question_elapsed_time'].fillna(X['prior_question_elapsed_time'].mean())

X = pd.merge(X, results_u, on=['user_id'], how="left")
X = pd.merge(X, results_c, on=['content_id'], how="left")
X = pd.merge(X, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')

X=X[X.answered_correctly!= -1 ]
X=X.sort_values(['user_id'])
Y = X[["answered_correctly"]]
X = X.drop(["answered_correctly"], axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
X['prior_question_had_explanation_enc'] = lb_make.fit_transform(X['prior_question_had_explanation'])
X.head()

X.fillna(0.5,inplace=True)

In [ ]:
X = X[['answered_correctly_user', 'answered_correctly_content', 'sum','bundle_id','part','prior_question_elapsed_time','prior_question_had_explanation_enc','tags1','tags2','tags3']]

In [ ]:
from sklearn.model_selection import train_test_split

Xt , Xv , Yt,Yv = train_test_split(X,Y,test_size=0.2,shuffle=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

### Further trim the dataset for faster iteration

In [ ]:
Xt_trim = Xt[75000:].reset_index(drop=True)
Yt_trim = Yt[75000:].reset_index(drop=True)

In [ ]:
Xv_trim = Xv[25000:].reset_index(drop=True)
Yv_trim = Yv[25000:].reset_index(drop=True)

# Model Interpretability 

In [ ]:
import xgboost as xgb

In [ ]:
data_dmatrix = xgb.DMatrix(data=Xt_trim.values,label=Yt_trim.values)

In [ ]:
params = {"objective":"binary:logistic",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10,"eval_metric": "auc"}

In [ ]:
#clf = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10) 
model = xgb.train(params, xgb.DMatrix(Xt_trim, label=Yt_trim),  num_boost_round=10)

# SHAP values

SHAP values interpret the impact of having a certain value for a given feature in comparison to the prediction we'd make if that feature took some baseline value.

In [ ]:
import shap 

# load JS visualization code to notebook
shap.initjs()

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(Xv_trim)

#use matplotlib=True


# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
shap.force_plot(explainer.expected_value, shap_values[0,:], Xv_trim.iloc[0,:])

### For first response in dataset have 12% chance of giving the correct answer 

### Model base value stands at 41.8% its need to be improved 

In [ ]:
Xv_trim.iloc[0,:]

In [ ]:
del train,X 
import gc 
gc.collect()

# Summarize the impact of all features over the entire dataset
A SHAP value for a feature of a specific prediction represents how much the model prediction changes when we observe that feature. In the summary plot below we plot all the SHAP values for a single feature on a row, where the x-axis is the SHAP value (which for this model is in units of log odds of giving correct answer to Question). By doing this for all features, we see which features drive the model's prediction a lot (such as Correct Answers), and which only effect the prediction a little (such as kills). Note that when points don't fit together on the line they pile up vertically to show density. Each dot is also colored by the value of that feature from high to low.

In [ ]:
shap_values = explainer.shap_values(Xv_trim[2:3])
shap.force_plot(explainer.expected_value, shap_values, Xv_trim[2:3])

## How do you interpret this?

We predicted 0.16 , whereas the base_value is 0.418. Feature values causing increased predictions are in pink, and their visual size shows the magnitude of the feature's effect. Feature values decreasing the prediction are in blue. The biggest impact comes from tag 4.

If you subtract the length of the blue bars from the length of the pink bars, it equals the distance from the base value to the output.

# Examine how changes in a feature change the model's prediction

## ( Feature Interaction with model output)

The XGBoost model we trained above is very complicated, but by plotting the SHAP value for a feature against the actual value of the feature for all players we can see how changes in the feature's value effect the model's output. Note that these plots are very similar to standard partial dependence plots, but they provide the added advantage of displaying how much context matters for a feature (or in other words how much interaction terms matter)

In [ ]:
# sort the features indexes by their importance in the model
# (sum of SHAP value magnitudes over the validation dataset)


explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(Xv_trim)


top_inds = np.argsort(-np.sum(np.abs(shap_values), 0))

# make SHAP plots of the three most important features
for i in range(len(top_inds)):
    shap.dependence_plot(top_inds[i], shap_values, Xv_trim)

<font color="red" size=3>Please upvote this kernel if you like it. It motivates me to create kernal with great content  :) </font>